# Scrape Amazon items Detail

- Technologies:  Requests, Beautifulsoup, Pandas, ExcelWriter, sqlite3

In [ ]:
from bs4 import BeautifulSoup
import requests
from pandas import ExcelWriter
import pandas as pd
import sqlite3
  

HEADERS = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'}
url='https://www.amazon.in/s?k=5g+mobile+phones+latest&crid=3A0EDR3F2S3HL&sprefix=mobil%2Caps%2C352&ref=nb_sb_ss_ts-doa-p_2_5'
webpage = requests.get(url, headers=HEADERS)
soup = BeautifulSoup(webpage.content, "lxml")

In [ ]:
data_asins=soup.find_all('div',{'data-index':True})
url='https://www.amazon.in/gp/product/'
asin_list=[]
for asin in data_asins:
    if asin['data-asin'] != '':
        asin_list.append(url + asin['data-asin'])
asin_list
len(asin_list)

In [ ]:
def get_mobile_webpage(url):
    webpage = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(webpage.content, "lxml")
    return soup
    

In [ ]:
def get_mobile_detail(mobile_url):
    soup=get_mobile_webpage(mobile_url)
    mobile_title=soup.find('h1',{'id':'title'})
    title=mobile_title.text.strip()[:12]
    
    mobile_price=soup.find('td',{'class':'a-span12'})
    price=mobile_price.text.strip()
    
    
    mobile_imgs=soup.find('div',{'id':'imgTagWrapperId'})
    imgurl=mobile_imgs.img['src']
    
    
        
    return title,price,imgurl

In [ ]:
get_mobile_detail(asin_list[6])

In [ ]:
def get_allMobiles():
    mobile_dict ={ 
        'ProductLinks': asin_list,     
        'Title':[][:17],
                 'Price': [],
            'ImageLinks': [],
           }

    # Get repo info
    for i in range(22):
        mobile_info = get_mobile_detail(asin_list[i])
        mobile_dict['Title'].append(mobile_info[0])
        mobile_dict['Price'].append(mobile_info[1])
        mobile_dict['ImageLinks'].append(mobile_info[2])
        
    return pd.DataFrame(mobile_dict)

## Saving to ExcelSheet

In [ ]:
writer = ExcelWriter('AmazonProducts.xlsx')
get_allMobiles().to_excel(writer,'Sheet5')
writer.save()

## Saving to Database

In [ ]:
conn = sqlite3.connect('mobilesDb.db')
c = conn.cursor()
c.execute('''CREATE TABLE Mobiles( ProductLinks TEXT, Title TEXT, Price TEXT, ImageLinks TEXT )''')
data=get_allMobiles()
data.to_sql('SA', conn, if_exists='replace', index=False) # - writes the pd.df to SQLIte DB
pd.read_sql('select * from Mobiles', conn)
conn.commit()
conn.close()

# Scraping Hidden/Data-asin Elements From Cart

- we can scrape product-detail from www.amazon.in using BeautifulSoup, but not from its cart yet!

- I have found some old Techniques to scrape data-asin elements with Beautifulsoup, but these are not working due to amazon updated privacy

- we can get data-asin elements by an API and use them to scrape detail

- we can Scrape detail using selenium-Webdriver, as u know i am not familiar with it, but i try to scrape detail with selenium, i will show u code in my task 

### Old  BeautifulSoup  Techniques

##### Technique 1

- Using BeautifulSoup, selenium-Webdriver and requests

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

driver=webdriver.Chrome('C://Windows/chromedriver.exe')
url='https://www.amazon.com/gp/cart/view.html?ref_=nav_cart'
driver.get(url)
soup=BeautifulSoup(driver.page_source,'html.parser')
# print(soup.prettify())

##### Technique 2

- Using requests_html and time

In [ ]:
from requests_html import HTMLSession
import time

s = HTMLSession()
url = 'https://barefootbuttons.com/product-category/version-1/'
r = s.get(url)
items = r.html.find('div.product-small.box')

### Scrape Amazon Products using Selenium

In [ ]:
from selenium import webdriver
from time import sleep

path = 'C://chromedriver.exe'
browser = webdriver.Chrome(executable_path=path)
browser.get('https://www.amazon.in')
browser.maximize_window()

input_search = browser.find_element_by_id('twotabsearchtextbox')
search_button = browser.find_element_by_xpath("(//input[@type='submit'])[1]")

input_search.send_keys("Smartphones under 10000")
sleep(1)
search_button.click()

products = []
for i in range(10):
    print('Scraping page', i+1)
    product = browser.find_elements_by_xpath("//span[@class='a-size-medium a-color-base a-text-normal']")
    for p in product:
        products.append(p.text)
    next_button = browser.find_element_by_xpath("//li[@class='a-last']")
    next_button.click()
    sleep(2)
    

brower.quit()